# Neural Network Weight Optimisation Using Ant Colony Optimisation

## Importing necessary libraries

In [1]:
import sklearn
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import random

2023-04-05 14:09:09.027987: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 14:09:09.554670: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Creating a neural network using tensorflow keras

In [44]:
model = Sequential([tf.keras.layers.Dense(12,input_shape = (12,),activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(16, activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(2,activation = 'softmax')
                   ])

In [45]:
model.compile(optimizer = 'adam', metrics = ['accuracy'],loss = 'categorical_crossentropy')

## Reading the dataset and splitting it into train and test data

In [4]:
df = pd.read_csv('processed_data.csv')

In [5]:
df = df.drop('Unnamed: 0',axis = 1)

In [6]:
df = pd.get_dummies(df,columns = ['Education'],drop_first = True)

In [7]:
X = df.drop('target',axis = 1)
y = df['target']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                    test_size=0.33, 
                                                    random_state=42,
                                                    shuffle = True)

### Converting target into categorical data

In [9]:
y_train=np_utils.to_categorical(y_train,num_classes=2)
y_test=np_utils.to_categorical(y_test,num_classes=2)

## Setting up some variables required for the ant colony algorithms

In [46]:
weights_shapes = [w.shape for w in model.get_weights()]

# Compute the total number of weights
num_weights = np.sum([np.prod(s) for s in weights_shapes])

l = num_weights

l = nh(ni + 1) + no(nh + 1)

In [47]:
v = 10

In [105]:
w_min = -1
w_max = 1

In [106]:
a_matrix = np.random.uniform(low = w_min, high = w_max,size = (v,l)) #functions as the graph for ants to traverse

In [107]:
t_matrix = np.ones((v,l)) #functions as the pheromone matrix

In [112]:
def roulette_wheel_selection(prob_list): #for the ants to choose a point based on the probabilities of choosing the next point
    """
    Selects an index from the given list of probabilities using roulette wheel selection.
    
    Args:
    prob_list: A list of non-negative floating point numbers representing probabilities.
    
    Returns:
    An integer representing the index selected.
    """
    # Compute the cumulative sum of probabilities
    cum_prob = np.cumsum(prob_list)
    
    # Generate a random number between 0 and the sum of probabilities
    rand_num = random.uniform(0, cum_prob[-1])
    
    # Find the index whose probability range encompasses the random number
    index = np.searchsorted(cum_prob, rand_num)
    
    return index

In [113]:
def choose_discrete_point(v,weight_idx): #creates the probabilities for choosing the point and chooses a point based on rws, returns the index of chosen point
    probabilities = []
    for point in range(v):
        prob = t_matrix[point][weight_idx]/t_matrix.sum(axis = 0)[weight_idx] #formula from research paper
        probabilities.append(prob)
    idx_of_selected_point = roulette_wheel_selection(probabilities)
    return (idx_of_selected_point)

In [123]:
def release_ants(number_of_ants,l,v): # creates a path for each ant and returns the list of paths
    ant_paths = list()
    for i in range(number_of_ants):
        path = list()
        for weight_idx in range(l):
            discrete_point = choose_discrete_point(v,weight_idx)
            path.append(discrete_point)
            #print(path)
        ant_paths.append(path)
    return ant_paths

In [115]:
def trace_path(number_of_ants,ant_paths,a_matrix,model): #based on the path traversed by ant, selects weights from a_matrix and sets it to the model and evaluated the model 
    errors = []
    accuracies = []
    for i in range (number_of_ants):
        point_indices = ant_paths[i]
        weights = []
        for weight, point in enumerate(point_indices):
            weights.append(a_matrix[point][weight])
        reshaped_weights = reshape_weights(weights, model)
        model.set_weights(weights = reshaped_weights)
        error,accuracy = model.evaluate(X_train,y_train)
        errors.append(error)
        accuracies.append(accuracy)
        
    return errors,accuracies

In [116]:
def retrace(errors,number_of_ants,ant_paths,t_matrix): #deposits pheromones to the path chosen by ant. the amount of pheromone deposited is calculated based on the error generated by the ant.
    for i in range (number_of_ants):
        delta_ph = 1/errors[i]
        path = ant_paths[i]
        for weight,idx in enumerate(path):
            t_matrix[idx][weight] += delta_ph

In [117]:
def decay(t_matrix, rate): #decays the pheromone matrix to prevent propogation of mistakes made in the past
    decayed = t_matrix * (1 - rate)
    return np.where(decayed < 0, 0, decayed)

In [118]:
def reshape_weights(weights, model): #takes an array of weights and reshapes it so that it can be used in set_weights function
    """
    Reshapes the weights from a 1D array to the shape required by a Keras model.
    
    Args:
    weights: A 1D array of weights.
    model: A Keras model whose weights are to be reshaped.
    
    Returns:
    A list of Numpy arrays containing the reshaped weights.
    """
    flattened_weights = np.array(weights)
    weights_shapes = [w.shape for w in model.get_weights()]

    # Compute the total number of weights
    num_weights = np.sum([np.prod(s) for s in weights_shapes])

    flattened_weights = np.array(weights)
    # Check that the total number of weights matches the length of the flattened list
    assert num_weights == len(flattened_weights)

    # Reshape the flattened list into a list of weight tensors with the same shapes as the model's weights
    weight_tensors = []
    idx = 0
    for shape in weights_shapes:
        size = np.prod(shape)
        weight_flat = flattened_weights[idx:idx+size]
        weight = tf.constant(np.array(weight_flat).reshape(shape))
        weight_tensors.append(weight)
        idx += size
    
    return weight_tensors

In [119]:
def aco(number_of_ants, l, v, t_matrix, model): 
    """"
    step 1: release ants - each ant chooses a value for each weight based on the given formula for probability for
            choosing a point and roulette wheel selection
    step 2: trace path - for each ant, set model weights and calculate errors 
    step 3: retrace - for each ant, calculate the pheromone to be deposited and add it to every point visited by it
    step 4: decay the whole pheromone matrix
    step 5: repeat the whole thing until convergence
    """   
    ant_paths  = release_ants(number_of_ants,l,v)
    errors,accuracies = trace_path(number_of_ants,ant_paths,a_matrix,model)
    retrace(errors, number_of_ants, ant_paths,t_matrix)
    t_matrix = decay(t_matrix, 0.1)
    return errors,accuracies,ant_paths
    

In [120]:
def set_best_weights(accuracies,model,ant_paths): #sets the best weight based on the ant which gives the model with the highest accuracy
    best_path_idx = accuracies.index(max(accuracies))
    best_path = ant_paths[best_path_idx]
    weights = []
    for weight, idx in enumerate(best_path):
        weights.append(a_matrix[idx][weight])
    flattened_weights = np.array(weights)
    weights_shapes = [w.shape for w in model.get_weights()]

    # Compute the total number of weights
    num_weights = np.sum([np.prod(s) for s in weights_shapes])
    flattened_weights = np.array(weights)
    
    assert num_weights == len(flattened_weights)

    # Reshape the flattened list into a list of weight tensors with the same shapes as the model's weights
    weight_tensors = []
    idx = 0
    for shape in weights_shapes:
        size = np.prod(shape)
        weight_flat = flattened_weights[idx:idx+size]
        weight = tf.constant(np.array(weight_flat).reshape(shape))
        weight_tensors.append(weight)
        idx += size
    model.set_weights(weight_tensors)

In [125]:
number_of_ants = 5
for i in range(10):
    print('Step:',i)
    errors,accuracies,ant_paths = aco(number_of_ants, l, v, t_matrix, model)
set_best_weights(accuracies,model,ant_paths)

Step: 0
92/92 [==============================] - 0s 444us/step - loss: 131.8449 - accuracy: 0.9366
Step: 1
92/92 [==============================] - 0s 429us/step - loss: 44.0643 - accuracy: 0.6036
Step: 2
92/92 [==============================] - 0s 445us/step - loss: 62.7310 - accuracy: 0.9366
Step: 3
92/92 [==============================] - 0s 441us/step - loss: 18.4211 - accuracy: 0.5793
Step: 4
92/92 [==============================] - 0s 442us/step - loss: 2.2014 - accuracy: 0.8287
Step: 5
92/92 [==============================] - 0s 480us/step - loss: 15.1536 - accuracy: 0.6632
Step: 6
92/92 [==============================] - 0s 477us/step - loss: 12.8984 - accuracy: 0.5913
Step: 7
92/92 [==============================] - 0s 438us/step - loss: 288.7115 - accuracy: 0.0634
Step: 8
92/92 [==============================] - 0s 437us/step - loss: 35.2657 - accuracy: 0.9130
Step: 9
92/92 [==============================] - 0s 451us/step - loss: 38.8399 - accuracy: 0.9366


In [126]:
model.evaluate(X_test, y_test)

45/45 [==============================] - 0s 496us/step - loss: 27.5587 - accuracy: 0.9437


[27.558746337890625, 0.9437109231948853]